In [1]:
from Crypto import Random
from Crypto.PublicKey import RSA
import base64

In [183]:
def toBase64(string):
    return base64.b64encode(string)

In [11]:
ori_key = RSA.generate(256*5,Random.new().read)

In [26]:
ori_key

RsaKey(n=16509972091301062925048533241059454173341151383094110438103714968760381972099601444544077834301452243450061772173844353310663944806713596637684402329300693919919659358069063349141087322823656178572043599563385297696452444112798842329254007589845135234285833418101025450100603157546027295619422555804073192411418387683857919567465999623834089911185697465769866269173369859495307119992487, e=65537, d=7159519154596277039380492160320272328705243180303257986342181964572227225034265728579927247979725540669404390881191640158827369446370758753726760672577715202162392525692704584930492724943898993912725226519565234250397092959160112528072135522323576170780268079824441815982463742520784679387744935952443187269697840017579474100669783626154877837306680732879279187622808655815768179261633, p=4038598884436711732555296226927435949668626111651880720242730596025682737548193817644375599485533740402765184778469937963383056392573856859107850179525681254317531395577655894006461573865361449, q=408804

**Testing keys**

In [181]:
from Crypto.Cipher import PKCS1_OAEP

pub_key = PKCS1_OAEP.new(ori_key.public_key())
encrypted = pub_key.encrypt(b'hey whatssup')


In [182]:
str.encode('hey')

b'hey'

In [85]:
prikey = PKCS1_OAEP.new(ori_key)
prikey.decrypt(encrypted)

b'hey whatssup'

**Sharding key**

In [187]:
class Shard(RSA.RsaKey):
    def __init__(self,x,n,d,e,q,p,u):
        RSA.RsaKey.__init__(self,n=n,d=d,e=e,q=q,p=p,u=u)
        self.x = x 
    def decode(self):
        return super().decode(self)

For 2 shards to be able to form the key, we must have all of the shards in a straight line that follows the rule of y=ax+b.
In here, 'x' and 'b' will randomized; and 'a' will be the hidden values like d,q,p and u. The shards will be given 'x' and 'y'.
2 Shards will be needed to compute 'a', which will be the needed attributes of the private key

In [44]:
#Determining the equation 
import random
b = random.randint(1,1000000)
#Getting a list of 5 random numbers from 1 to 100
li = range(1,1000001)
x = random.sample(li,5)
#Printing equations:
print("The equations are:")
for i in range(1,6):
    print("x{:d}: a*{:d} + {:d}".format(i,x[i-1],b))


The equations are:
x1: a*973062 + 394297
x2: a*961972 + 394297
x3: a*651937 + 394297
x4: a*842076 + 394297
x5: a*61270 + 394297


In [161]:
def sharding_linear(nums,private_key):
    if isinstance(private_key,RSA.RsaKey):
        b = random.randint(1,1000000)
        li = range(1,1000001)
        x = random.sample(li,nums)
        list = []
        for i in range(0,nums):
            n = private_key.n
            d = private_key.d * x[i] + b
            e = private_key.e
            q = private_key.q * x[i] + b 
            p = private_key.p * x[i] + b 
            u = private_key.u * x[i] + b 
            list.append(Shard(x=x[i],n=n,d=d,e=e,q=q,p=p,u=u))
        return list 
    else:
        print("Key is not RSA key")

    
    

In [162]:
shards = sharding_linear(5,ori_key)
shards[0]

RsaKey(n=16509972091301062925048533241059454173341151383094110438103714968760381972099601444544077834301452243450061772173844353310663944806713596637684402329300693919919659358069063349141087322823656178572043599563385297696452444112798842329254007589845135234285833418101025450100603157546027295619422555804073192411418387683857919567465999623834089911185697465769866269173369859495307119992487, e=65537, d=664031082550495502848461886885384617942753894486766571717264692850144930667478077794331092395623584446005918445448762241450920861411995132890649598860237929570157581972946964843133339253096743887417439309236636346255829577776182116753634425424467042687528303867557329548741547191317737443854567319717200732889935265950461063888921091758612609654520024613087386093640257209600867150070937484, p=374571969333736139771038614455065829459865734603488633041072777320190022542119880198880548101084283354875665357833529806227851714298440075968534888450647884975442401877036428857311298052924456671852

In [189]:
shards[2]

ValueError: No inverse value can be computed12

In [164]:
type(shards[0].x)

int

**Combining shards to find key**

In [172]:
def combine_linear_shards(shard1,shard2):
    #Solving linear equation
    d_difference = shard1.d-shard2.d
    q_difference = shard1.q - shard2.q
    p_difference = shard1.p - shard2.p
    u_difference = shard1.u - shard2.u
    x_difference = shard1.x-shard2.x
    d = d_difference // x_difference
    q = q_difference // x_difference
    p = p_difference // x_difference
    u = u_difference // x_difference
    
    return RSA.RsaKey(d=d,n=shard1.n,e=shard1.e,q=q,p=p,u=u)
    
    

In [174]:
combined = combine_linear_shards(shards[0],shards[2])

**Testing shards and keys**

In [176]:
#Try out shards
shard1 = PKCS1_OAEP.new(shards[0])
shard1.decrypt(encrypted)

ValueError: Fault detected in RSA decryption

In [175]:
#Try out combined key
combined_key = PKCS1_OAEP.new(combined)
combined_key.decrypt(encrypted)

b'hey whatssup'